In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
import pandas as pd

In [ ]:
import simplejson


def json_save(data, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        simplejson.dump(data, f, separators=(',', ':'), iterable_as_array=True)

In [ ]:
df_train = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')

In [ ]:
# # dev
# df_train = df_train.sample(50, random_state=42)
# df_test = df_test.sample(50, random_state=42)

In [ ]:
df_train.shape, df_test.shape

In [ ]:
from tqdm.notebook import tqdm_notebook

In [ ]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

def corpus2tokens(raw_corpus):
    return [
        [
            wd
            for wd in simple_preprocess(doc)
            if not wd in STOPWORDS
        ]
        for doc in tqdm_notebook(raw_corpus)
    ]

In [ ]:
%%time
corpus_words_train = corpus2tokens(df_train.text.to_list())

In [ ]:
json_save(corpus_words_train, './lda.corpus.train.json')

In [ ]:
%%time
from gensim import corpora

dictionary = corpora.Dictionary(corpus_words_train)
dictionary.filter_extremes()
dictionary.save_as_text('./lda.dictionary.txt')

In [ ]:
%%time
corpus_train = [dictionary.doc2bow(text) for text in corpus_words_train]
corpora.MmCorpus.serialize('./lda.corpus.txt', corpus_train)

In [ ]:
from gensim import models

In [ ]:
%%time
from gensim import models

lda = models.ldamulticore.LdaMulticore(
    corpus_train,
    id2word=dictionary, 
    num_topics=20,
    chunksize=4000,
    passes=20,
    workers=63
)
lda.save("./lda.model")

In [ ]:
%%time
text_test = corpus2tokens(df_test.text.to_list())

In [ ]:
json_save(text_test, './lda.corpus.test.json')